In [1]:
print("Welcome to my EMR Notebook!")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
25,application_1597998755054_0047,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Welcome to my EMR Notebook!

In [3]:
# Import libraries
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, \
                                  avg, from_unixtime, split, min, max, round, lit, mean
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
import datetime
from pyspark.sql.functions import to_date, year, month, dayofmonth, dayofweek, hour, date_format, substring
import numpy as np
import time

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Create spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Set time parser policy
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Read in full sparkify dataset
event_data = "s3n://udacity-dsnd/sparkify/sparkify_event_data.json"
data = spark.read.json(event_data)
data.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(artist='Popol Vuh', auth='Logged In', firstName='Shlok', gender='M', itemInSession=278, lastName='Johnson', length=524.32934, level='paid', location='Dallas-Fort Worth-Arlington, TX', method='PUT', page='NextSong', registration=1533734541000, sessionId=22683, song='Ich mache einen Spiegel - Dream Part 4', status=200, ts=1538352001000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1749042')

In [7]:
# cancelled users count
data.where(data.page == 'Cancellation Confirmation').select('userId').dropDuplicates().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5003

In [6]:
# downgraded users count
data.where(data.page == 'Submit Downgrade').select('userId').dropDuplicates().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5103

##### Functions

In [8]:

# Function that returns all users of a specified churn group
def get_users(churn):
    return data.where(data.churn == 1).select('userId').dropDuplicates()

# Count page logs
def page_count(page):
    return page_data \
        .where(data.page == page) \
        .groupby('userId') \
        .count() \
        .select('userId', col('count').alias(page.replace(' ', '') + 'Count'))

# Average page count per session hour
def page_session_hour(page):
    return page_data \
        .where(data.page == page) \
        .join(session_hours, ['userId', 'sessionId'], 'inner') \
        .groupby( 'userId', 'sessionId', 'sessionHours') \
        .agg((count('userId')/col('sessionHours')).alias('avgPerSession')) \
        .groupby('userId') \
        .agg(avg('avgPerSession').alias('avg')) \
        .select('userId', col('avg').alias(page.replace(' ', '') + 'PerSessionHour'))

# Average page count per hour
def page_hour(page):
    return page_data \
        .where(data.page == page) \
        .join(user_hours, 'userId', 'inner') \
        .groupby('userId', 'hours') \
        .agg((count('userId')/col('hours')).alias('avg')) \
        .select('userId', col('avg').alias(page.replace(' ', '') + 'PerHour'))

# Average page count per day
def page_day(page):
    return page_data \
        .where(data.page == page) \
        .groupby('userId', 'date') \
        .count() \
        .groupby('userId') \
        .agg(avg('count').alias(page.replace(' ', '') + 'PerDay'))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### Prepare data

In [ ]:

# Remove rows with missing users
data = data.where(~((col('userId').isNull()) | (col('userId') == '')))

# Exclude non-relevant columns
data = data.drop('firstName')
data = data.drop('lastName')

# Add tsDate and date column
data = data.withColumn('tsDate', (col('ts') / 1000.0).cast(TimestampType()))
data = data.withColumn('date', date_format(col('tsDate'), 'yyyy-MM-dd').alias('date').cast('date'))

# Label churned users using Submit Downgrade event
query_churn_by_cc = data.where(data.page == 'Submit Downgrade')
canceled = query_churn_by_cc.select('userId').dropDuplicates().select('userId')
canceled_uids = [row.userId for row in canceled.collect()];
set_churn = udf(lambda x: 1 if x in canceled_uids else 0, IntegerType())
data = data.withColumn('churn', set_churn('userId'))

# Add [userRowId] column that assigns a 1-based index to every user's log ordered by [ts]
w =  Window.partitionBy(data.userId).orderBy('ts', 'itemInSession')
data = data.withColumn('userRowId', row_number().over(w))

# Add [userRowDescId] column that assigns a 1-based index to every user's log ordered by [ts] descending.
w =  Window.partitionBy(data.userId).orderBy(col('ts').desc(), col('itemInSession').desc())
data = data.withColumn('userRowDescId', row_number().over(w))

# Add last level column
last_levels = dict()
for row in data.where(data.userRowDescId == 1).select('userId', 'level').collect():
    last_levels[row.userId] = row.level
get_level = udf(lambda userId: last_levels[userId])
data = data.withColumn('lastLevel', get_level('userId'))

# Prepare labels
labels = data.select(col('churn').alias('label'), 'userId').dropDuplicates()

# Check churned users
get_users(1).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5103

In [30]:
data.select('churn', 'userId').dropDuplicates().groupby('churn').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-----+
|churn|count|
+-----+-----+
|    1| 5103|
|    0|17175|
+-----+-----+

In [10]:
data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- tsDate: timestamp (nullable = true)
 |-- date: date (nullable = true)
 |-- churn: integer (nullable = true)
 |-- userRowId: integer (nullable = true)
 |-- userRowDescId: integer (nullable = true)
 |-- lastLevel: string (nullable = true)

##### Queries

In [19]:

# All unique users
users = data.select('userId').dropDuplicates()

# Pages without churn definition events
page_data = data.where(~data.page.isin(['Submit Downgrade', 'Upgrade', 'Submit Upgrade', 'Cancel', 'Cancellation Confirmation'])) \
    .select('churn', 'page', 'userId', 'sessionId', 'ts', 'date')

# Calc session duration (in hours)
session_hours = page_data \
    .groupby('userId', 'sessionId') \
    .agg(((max('ts') - min('ts'))/1000/3600).alias('sessionHours'))

# User interactions duration per user (in hours)
user_hours = page_data \
    .groupby('userId', 'sessionId') \
    .agg(((max('ts') - min('ts'))/1000/3600).alias('sessionHours')) \
    .groupby('userId') \
    .agg(Fsum('sessionHours').alias('hours'))  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Features

In [12]:
    f_Gender = data \
        .select('userId', 'gender') \
        .dropDuplicates() \
        .replace(['M', 'F'], ['0', '1'], 'gender') \
        .select('userId', col('gender').cast('int').alias('Gender'))

    f_LastLevel = data \
        .select('userId', 'lastLevel') \
        .dropDuplicates() \
        .replace(['free', 'paid'], ['0', '1'], 'lastLevel') \
        .select('userId', col('lastLevel').cast('int').alias('LastLevel'))

    f_LogCount = data \
        .groupby('userId') \
        .agg(count('userId').alias('LogCount'))

    f_SongCount = data \
        .where(data.page == 'NextSong') \
        .groupby('userId') \
        .agg(count('userId').alias('SongCount'))

    f_NonSongCount = data \
        .where(data.page != 'NextSong') \
        .groupby('userId') \
        .agg(count('userId').alias('NonSongCount'))

    f_AboutCount = page_count('About')

    f_ThumbsUpCount = page_count('Thumbs Up')

    f_RollAdvertCount = page_count('Roll Advert')

    f_SessionCount = data \
        .select('userId', 'sessionId') \
        .dropDuplicates() \
        .groupby('userId') \
        .agg(count('userId').alias('SessionCount'))

    f_AvgSessionLength = data \
       .groupby('userId', 'sessionId') \
       .agg(((max('ts') - min('ts'))/1000).alias('sessionLength')) \
       .groupby('userId') \
       .agg(avg('sessionLength').alias('AvgSessionLength')) \

    f_AvgSessionGap = data \
        .groupby('userId', 'sessionId') \
        .agg(min('ts').alias('startTime'), max('ts').alias('endTime')) \
        .groupby('userId') \
        .agg(count('userId').alias('sessionCount'), \
            ((max('endTime') - min('startTime'))/1000).alias('observationPeriodTime'), \
            (Fsum(col('endTime') - col('startTime'))/1000).alias('totalSessionTime')) \
        .where(col('sessionCount') > 1) \
        .join(users, 'userId', 'outer') \
        .fillna(0) \
        .select('userId', \
            (col('observationPeriodTime') - col('totalSessionTime')/(col('sessionCount') - 1)).alias('AvgSessionGap'))

    f_DowngradePerSessionHour = page_session_hour('Downgrade')

    f_ErrorPerSessionHour = page_session_hour('Error')

    f_SettingsPerSessionHour = page_session_hour('Settings')

    f_SaveSettingsPerSessionHour = page_session_hour('Save Settings')

    f_LogoutPerSessionHour = page_session_hour('Logout')

    f_SubmitDowngradePerSessionHour = page_session_hour('Submit Downgrade')

    f_RollAdvertPerHour = page_hour('Roll Advert')

    f_ThumbsDownPerHour = page_hour('Thumbs Down')

    f_UpgradePerHour = page_hour('Upgrade')

    f_SubmitUpgradePerHour = page_hour('Submit Upgrade')

    f_SessionsPerDay = data \
        .select('userId', 'date', 'sessionId') \
        .dropDuplicates() \
        .groupby('userId', 'date') \
        .count() \
        .groupby('userId') \
        .agg(avg('count').alias('SessionsPerDay'))

    f_AddFriendPerDay = page_day('Add Friend')

    f_RollAdvertPerDay = page_day('Roll Advert')

    f_ThumbsDownPerDay = page_day('Thumbs Down')

    f_ThumbsUpPerDay = page_day('Thumbs Up')

    f_TotalSongLength = data \
        .where(data.page == 'NextSong') \
        .select('userId', 'length') \
        .groupby('userId') \
        .agg(Fsum('length').alias('TotalSongLength'))

    f_UniqueSongCount = data \
        .where(data.page == 'NextSong') \
        .select('userId', 'song') \
        .dropDuplicates() \
        .groupby('userId') \
        .agg(count('userId').alias('UniqueSongCount'))

    f_UniqueSongShare = data \
        .where(data.page == 'NextSong') \
        .select('userId', 'song') \
        .dropDuplicates() \
        .groupby('userId') \
        .count() \
        .join(f_SongCount, on = ['userId'], how = 'inner') \
        .select('userId', (col('count')/col('SongCount')).alias('UniqueSongShare')) 


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Feature Check

### AvgSessionGap (YES)

In [13]:
labels.join(f_AvgSessionGap, 'userId', 'outer') \
    .groupby('label') \
    .agg(avg('AvgSessionGap')) \
    .show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------------------+
|label|avg(AvgSessionGap)|
+-----+------------------+
|    1| 4244186.260620306|
|    0| 3304447.189541696|
+-----+------------------+

In [20]:
labels.join(f_AvgSessionGap, 'userId', 'outer') \
    .groupby('label') \
    .agg(avg('AvgSessionGap')) \
    .show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------------------+
|label|avg(AvgSessionGap)|
+-----+------------------+
|    1| 2083145.432607491|
|    0| 3935744.507611506|
+-----+------------------+

### Per hour features

In [16]:
# Average page count per hour, per churn
page_data \
    .join(user_hours, 'userId', 'inner') \
    .groupby('churn', 'userId', 'page', 'hours') \
    .agg((count('userId')/col('hours')).alias('CountPerHour')) \
    .groupby('churn', 'page') \
    .agg(avg('CountPerHour').alias('AvgCountPerHour')) \
    .sort('page', 'churn') \
    .show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+--------------------+
|churn|                page|     AvgCountPerHour|
+-----+--------------------+--------------------+
|    0|               About| 0.07974660299961202|
|    1|               About|0.038796560736421515|
|    0|          Add Friend|  0.3323238283105589|
|    1|          Add Friend| 0.26757162138438756|
|    0|     Add to Playlist|  0.4307492174289696|
|    1|     Add to Playlist|  0.4065724958388631|
|    0|              Cancel| 0.27058577119877714|
|    1|              Cancel|0.021713037494027587|
|    0|Cancellation Conf...| 0.27058577119877714|
|    1|Cancellation Conf...|0.021713037494027587|
|    0|           Downgrade| 0.15553974027070466|
|    1|           Downgrade| 0.14083388826729795|
|    0|               Error| 0.04366372583976314|
|    1|               Error|  0.0239287671949968|
|    0|                Help|   0.121046621770175|
|    1|                Help| 0.09276112061952985|
|    0|                Home|  1.3876360760280084|


##### Selected per hour features:

 - Downgrade
 - Home
 - Logout
 - Roll Advert
 - Settings
 - Submit Downgrade
 - Submit Upgrade
 - Thumbs Down

### Per day features

In [17]:
# Average page count per day, per churn
page_data \
    .groupby('churn', 'userId', 'page', 'date') \
    .count() \
    .groupby('churn', 'userId', 'page') \
    .agg(avg('count').alias('AvgCountPerDay')) \
    .groupby('churn', 'page') \
    .agg(avg('AvgCountPerDay').alias('AvgCountPerDay')) \
    .sort('page', 'churn') \
    .show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+------------------+
|churn|                page|    AvgCountPerDay|
+-----+--------------------+------------------+
|    0|               About|1.2359981623425653|
|    1|               About|1.1769044524425163|
|    0|          Add Friend|2.4145275202506804|
|    1|          Add Friend| 2.663828774404083|
|    0|     Add to Playlist|   2.5970465586921|
|    1|     Add to Playlist|3.0302791793537764|
|    0|              Cancel|               1.0|
|    1|              Cancel|               1.0|
|    0|Cancellation Conf...|               1.0|
|    1|Cancellation Conf...|               1.0|
|    0|           Downgrade|1.8330022091560143|
|    1|           Downgrade|1.9526806700667658|
|    0|               Error|1.0759669936571994|
|    1|               Error|1.0808030144777132|
|    0|                Help| 1.433259928460583|
|    1|                Help|1.4588501845064854|
|    0|                Home|  3.52374669352495|
|    1|                Home|3.8617029426

##### Selected per day features:

 - RollAdvert

### SongCount (NO)

In [18]:
labels.join(f_SongCount, 'userId', 'outer') \
    .fillna(0) \
    .groupby('label').agg(avg('SongCount')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------------------+
|label|    avg(SongCount)|
+-----+------------------+
|    1|1748.1493239271017|
|    0|  694.583173216885|
+-----+------------------+

### NonSongCount (YES)

In [20]:
labels.join(f_NonSongCount, 'userId', 'outer') \
    .fillna(0) \
    .groupby('label').agg(avg('NonSongCount')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-----------------+
|label|avg(NonSongCount)|
+-----+-----------------+
|    1| 386.071526553008|
|    0|200.2214847161572|
+-----+-----------------+

### SessionCount (YES)

In [21]:
labels.join(f_SessionCount, 'userId', 'outer') \
    .fillna(0) \
    .groupby('label').agg(avg('SessionCount')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------------------+
|label| avg(SessionCount)|
+-----+------------------+
|    1| 23.54830491867529|
|    0|19.505735080058223|
+-----+------------------+

### AvgSessionLength (NO)

In [22]:
labels.join(f_AvgSessionLength, 'userId', 'outer') \
    .fillna(0) \
    .groupby('label').agg(avg('AvgSessionLength')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+---------------------+
|label|avg(AvgSessionLength)|
+-----+---------------------+
|    1|    18895.01848812512|
|    0|   15908.077736444633|
+-----+---------------------+

### TotalSongLength (NO)

In [23]:
labels.join(f_TotalSongLength, 'userId', 'outer') \
    .fillna(0) \
    .groupby('label').agg(avg('TotalSongLength')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+
|label|avg(TotalSongLength)|
+-----+--------------------+
|    1|   434794.7743775052|
|    0|    172764.785853936|
+-----+--------------------+

### SessionsPerDay (NO)

In [24]:
labels.join(f_SessionsPerDay, 'userId', 'outer') \
    .fillna(0) \
    .groupby('label').agg(avg('SessionsPerDay')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------------------+
|label|avg(SessionsPerDay)|
+-----+-------------------+
|    1| 1.2262613774098055|
|    0| 1.3028645682426432|
+-----+-------------------+

### UniqueSongCount (NO)

In [25]:
labels.join(f_UniqueSongCount, 'userId', 'outer') \
    .fillna(0) \
    .groupby('label').agg(avg('UniqueSongCount')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+
|label|avg(UniqueSongCount)|
+-----+--------------------+
|    1|  1501.8208896727415|
|    0|   628.5352547307133|
+-----+--------------------+

### UniqueSongShare (NO)

In [26]:
labels.join(f_UniqueSongShare, 'userId', 'outer') \
    .fillna(0) \
    .groupby('label').agg(avg('UniqueSongShare')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+
|label|avg(UniqueSongShare)|
+-----+--------------------+
|    1|   0.902333929373722|
|    0|  0.9504775922825538|
+-----+--------------------+

### LogCount (NO)

In [27]:
labels.join(f_LogCount, 'userId', 'outer') \
    .fillna(0) \
    .groupby('label').agg(avg('LogCount')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------------------+
|label|     avg(LogCount)|
+-----+------------------+
|    1|2134.2208504801097|
|    0| 894.8046579330422|
+-----+------------------+

### Per session-hour features

In [28]:
page_data \
    .join(session_hours, ['userId', 'sessionId'], 'inner') \
    .groupby('churn', 'page', 'userId', 'sessionId', 'sessionHours') \
    .agg((count('userId')/col('sessionHours')).alias('AvgPerSessionHour')) \
    .groupby('churn', 'page', 'userId') \
    .agg(avg('AvgPerSessionHour').alias('AvgPerSessionHour')) \
    .groupby('churn', 'page') \
    .agg(avg('AvgPerSessionHour').alias('AvgPerSessionHour')) \
    .sort('page', 'churn') \
    .show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------------------+-------------------+
|churn|               page|  AvgPerSessionHour|
+-----+-------------------+-------------------+
|    0|              About| 1.3444645752997937|
|    1|              About|  1.623732947278581|
|    0|         Add Friend| 11.101005878338443|
|    1|         Add Friend|  9.002988046530247|
|    0|    Add to Playlist| 0.8910132604671539|
|    1|    Add to Playlist| 0.8612317970890859|
|    0|          Downgrade| 0.7068792406841425|
|    1|          Downgrade| 0.5054197692132388|
|    0|              Error| 0.8401778912640101|
|    1|              Error|0.35152059539334196|
|    0|               Help| 1.2071021264084552|
|    1|               Help|  1.570043252059434|
|    0|               Home| 13.293379077967844|
|    1|               Home| 12.174322714941066|
|    0|              Login|   916.384243862892|
|    0|             Logout|  21.58056847479438|
|    1|             Logout| 19.548823872578662|
|    0|           NextSong| 28.963111939

##### Selected per day features:

 - About
 - Downgrade
 - Error
 - Settings
 - Upgrade

### Page count features

In [29]:
page_data \
    .groupby('churn', 'page', 'userId') \
    .count() \
    .groupby('churn', 'page') \
    .agg(avg('count').alias('AvgPageCount')) \
    .sort('page', 'churn') \
    .show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------------------+------------------+
|churn|               page|      AvgPageCount|
+-----+-------------------+------------------+
|    0|              About|7.1716640190627485|
|    1|              About| 4.676691729323308|
|    0|         Add Friend|14.336133902198885|
|    1|         Add Friend|32.199802761341225|
|    0|    Add to Playlist| 21.25598663449044|
|    1|    Add to Playlist| 49.89272406354187|
|    0|          Downgrade| 9.738670096972095|
|    1|          Downgrade|16.817754262198708|
|    0|              Error| 2.030910055717697|
|    1|              Error|2.8522088353413655|
|    0|               Help| 7.507071922961179|
|    1|               Help| 11.17043618739903|
|    0|               Home| 55.55833824395993|
|    1|               Home| 78.43954536547129|
|    0|              Login|          296350.0|
|    0|             Logout| 10.61476735506345|
|    1|             Logout|24.658143194335167|
|    0|           NextSong| 695.2713602984031|
|    1|      

### ML - Random Forest Test

In [78]:
# Import ML libraries
from pyspark.ml.feature import VectorAssembler, Normalizer, StandardScaler, MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
import numpy as np

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
columns = ['label', 'col1', 'col2', 'col3']
values = [[0, 0, 4, 7], [0, 0, 50, 19], [0, 0, 49, 2], [0, 0, 51, 6], [0, 1, 50, 99], \
          [1, 1, 22, 77], [1, 1, 97, 13], [1, 1, 89, 4], [1, 1, 98, 39], [1, 0, 93, 12],
          [1, 1, 101, 7], [1, 1, 105, 53], [1, 1, 96, 67], [1, 1, 100, 81], [1, 0, 105, 2]]
rdd = spark.sparkContext.parallelize(values)
df = rdd.toDF(columns) 
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----+----+----+
|label|col1|col2|col3|
+-----+----+----+----+
|    0|   0|   4|   7|
|    0|   0|  50|  19|
|    0|   0|  49|   2|
|    0|   0|  51|   6|
|    0|   1|  50|  99|
|    1|   1|  22|  77|
|    1|   1|  97|  13|
|    1|   1|  89|   4|
|    1|   1|  98|  39|
|    1|   0|  93|  12|
|    1|   1| 101|   7|
|    1|   1| 105|  53|
|    1|   1|  96|  67|
|    1|   1| 100|  81|
|    1|   0| 105|   2|
+-----+----+----+----+

In [80]:
# Return selected features without label  
def get_features(data):
    return data.drop('label').columns

# Split data into training and testing subset
def split_data(data, seed=0):
    train, test = data.randomSplit([0.8, 0.2], seed=seed);
    return train, test

# Create ML pipeline
def create_pipeline(data, classifier, scaler):
    features = get_features(data)
    assembler = VectorAssembler(inputCols=features, outputCol='NumFeatures')
    pipeline = Pipeline(stages=[assembler, scaler, classifier]);
    return pipeline

# Fit and evaluate model
def fit_model(data, classifier, seed=0):
    train, test = split_data(data, seed)
    scaler = StandardScaler(inputCol='NumFeatures', outputCol='features')
    pipeline = create_pipeline(data, classifier, scaler)
    model = pipeline.fit(train)
    return model

def calc_metrics(model, test):
    # Create both evaluators
    evaluatorMulti = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction')
    evaluator = BinaryClassificationEvaluator(labelCol='label', rawPredictionCol='prediction', metricName='areaUnderROC')

    # Make predicitons
    prediction = model.transform(test).select('label', 'prediction')

    # Get metrics
    accuracy = evaluatorMulti.evaluate(prediction, {evaluatorMulti.metricName: 'accuracy'})
    f1 = evaluatorMulti.evaluate(prediction, {evaluatorMulti.metricName: 'f1'})
    weightedPrecision = evaluatorMulti.evaluate(prediction, {evaluatorMulti.metricName: 'weightedPrecision'})
    weightedRecall = evaluatorMulti.evaluate(prediction, {evaluatorMulti.metricName: 'weightedRecall'})
    auc = evaluator.evaluate(prediction)

    # Write metrics to S3
    columns = ['f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC']
    values = [f1, accuracy, weightedPrecision, weightedRecall, auc]
    rdd = spark.sparkContext.parallelize([values])
    metrics = rdd.toDF(columns)
    
    return metrics

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
def get_feature_importances()

In [81]:
classifier = RandomForestClassifier(seed=0)
model = fit_model(df, classifier)
train, test = split_data(df)
metrics = calc_metrics(model, test)
metrics.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------+------------------+--------------+----+
|                f1|accuracy| weightedPrecision|weightedRecall| AUC|
+------------------+--------+------------------+--------------+----+
|0.7333333333333334|    0.75|0.8333333333333333|          0.75|0.75|
+------------------+--------+------------------+--------------+----+

In [82]:
values = []
features = list(get_features(df))
importances = list(model.stages[-1].featureImportances)
for x in list(zip(features, importances)):
    values.append([x[0], float(x[1])])
columns = ['feature', 'importance']
rdd = spark.sparkContext.parallelize(values)
df2 = rdd.toDF(columns) 
df2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+
|feature|         importance|
+-------+-------------------+
|   col1|0.19961104812295288|
|   col2| 0.5459124723013612|
|   col3|  0.254476479575686|
+-------+-------------------+

In [83]:
df2.write.mode('overwrite').json('s3n://amosvoron-sparkify/rfc-feature-importances-test.json')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Users

In [87]:
get_users(1).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5003

In [88]:
get_users(0).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '404' from https://172.31.39.107:18888/sessions/8/statements/88 with error payload: {"msg":"Session '8' not found."}


In [1]:
data.where(data.page == 'Submit Downgrade').select('userId').dropDuplicates().count()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
11,application_1597998755054_0026,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'data' is not defined
Traceback (most recent call last):
NameError: name 'data' is not defined



# Check metrics & feature importances

In [4]:
metrics = spark.read.json('s3n://amosvoron-sparkify/rfc-metrics-f22.json')
metrics.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+-----------------+------------------+------------------+
|                f1|          accuracy|weightedPrecision|    weightedRecall|               AUC|
+------------------+------------------+-----------------+------------------+------------------+
|0.8386651060216977|0.8577847674679847|0.858875141192138|0.8577847674679848|0.7055267657733111|
+------------------+------------------+-----------------+------------------+------------------+

In [9]:
importances = spark.read.json('s3n://amosvoron-sparkify/rfc-feature-importances-f22.json')
importances.sort(desc('importance')).show(30, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+---------------------+
|feature                |importance           |
+-----------------------+---------------------+
|AvgSessionGap          |0.531326926781188    |
|RollAdvertPerHour      |0.07203686762635017  |
|DowngradePerHour       |0.06676894742434188  |
|SessionCount           |0.062376199033023286 |
|NonSongCount           |0.050628344100157416 |
|RollAdvertPerDay       |0.04571131362963781  |
|RollAdvertCount        |0.03022773918358667  |
|DowngradePerSessionHour|0.023059199682565503 |
|ThumbsDownPerHour      |0.021905197950220757 |
|HomeCount              |0.01935741525646098  |
|LogoutPerHour          |0.014225091004298893 |
|ThumbsUpCount          |0.014185526067462045 |
|SettingsPerSessionHour |0.011980284038583023 |
|AboutCount             |0.00900477412830605  |
|SubmitUpgradePerHour   |0.006285279375803763 |
|SettingsPerHour        |0.005847270443837536 |
|SubmitDowngradePerHour |0.004204600333886947 |
|UpgradePerSessionHour  |0.0030743231489

In [ ]:
+-----------------------+---------------------+
|feature                |importance           |
+-----------------------+---------------------+
|AvgSessionGap          |0.531326926781188    |
|RollAdvertPerHour      |0.07203686762635017  |
|DowngradePerHour       |0.06676894742434188  |
|SessionCount           |0.062376199033023286 |
|NonSongCount           |0.050628344100157416 |
|RollAdvertPerDay       |0.04571131362963781  |
|RollAdvertCount        |0.03022773918358667  |
|DowngradePerSessionHour|0.023059199682565503 |
|ThumbsDownPerHour      |0.021905197950220757 |

In [4]:
0.531326926781188+0.07203686762635017+0.06676894742434188+0.062376199033023286+\
0.050628344100157416+0.04571131362963781+0.03022773918358667+\
0.023059199682565503+0.021905197950220757

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.9040407354110714

In [8]:
0.531326926781188+0.07203686762635017+0.06676894742434188+0.062376199033023286+\
0.050628344100157416+0.04571131362963781

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8288485985946985

In [ ]:
metrics = spark.read.json('s3n://amosvoron-sparkify/rfc-metrics-f14.json')
metrics.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

KeyboardInterrupt: 

In [4]:
importances = spark.read.json('s3n://amosvoron-sparkify/rfc-feature-importances-f14.json')
importances.sort(desc('importance')).show(30, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+--------------------+
|feature                |importance          |
+-----------------------+--------------------+
|AvgSessionGap          |0.5503473401971377  |
|SessionCount           |0.0755328970855733  |
|DowngradePerSessionHour|0.06387767719603579 |
|RollAdvertPerDay       |0.058078969317302276|
|DowngradePerHour       |0.055567250509668716|
|NonSongCount           |0.04031488073809684 |
|HomeCount              |0.038644683408958186|
|RollAdvertPerHour      |0.03527247959916269 |
|RollAdvertCount        |0.026382226805606757|
|ThumbsUpCount          |0.021947392718465943|
|ThumbsDownPerHour      |0.020575687568754056|
|SettingsPerSessionHour |0.005003693133466566|
|AboutCount             |0.00454000079526978 |
|LogoutPerHour          |0.003914820926501557|
+-----------------------+--------------------+

In [5]:
metrics = spark.read.json('s3n://amosvoron-sparkify/rfc-downgraded-metrics-22h.json')
metrics.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+------------------+-----------------+------------------+
|                f1|          accuracy| weightedPrecision|   weightedRecall|               AUC|
+------------------+------------------+------------------+-----------------+------------------+
|0.8137938477161866|0.8303751965850371|0.8179372014878397|0.830375196585037|0.6879226254158419|
+------------------+------------------+------------------+-----------------+------------------+

In [6]:
importances = spark.read.json('s3n://amosvoron-sparkify/rfc-downgraded-feature-importances-22h.json')
importances.sort(desc('importance')).show(30, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+---------------------+
|feature               |importance           |
+----------------------+---------------------+
|RollAdvertCount       |0.17318133456281243  |
|NonSongCount          |0.14980679335444136  |
|LogoutCount           |0.14308597905735157  |
|RollAdvertPerHour     |0.11810976830081082  |
|ThumbsDownCount       |0.07681389953808654  |
|LogCount              |0.06305414713381112  |
|TotalSongLength       |0.06300524317998007  |
|UniqueSongCount       |0.06166385885128488  |
|SongCount             |0.056957357621509525 |
|AddtoPlaylistCount    |0.023031310261460342 |
|HomePerHour           |0.017639635737101884 |
|SettingsCount         |0.013962993303425146 |
|AddFriendCount        |0.012300844044906415 |
|ThumbsUpCount         |0.006417285783558138 |
|LogoutPerHour         |0.004784251904021001 |
|SettingsPerSessionHour|0.0030380538126538576|
|AboutCount            |0.0030040309961354918|
|SaveSettingsCount     |0.002693362247583948 |
|SaveSettings

In [4]:
metrics = spark.read.json('s3n://amosvoron-sparkify/rfc-downgraded-metrics-13h.json')
metrics.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+------------------+------------------+------------------+
|                f1|          accuracy| weightedPrecision|    weightedRecall|               AUC|
+------------------+------------------+------------------+------------------+------------------+
|0.8243558706304979|0.8346439002471355|0.8235744592195376|0.8346439002471354|0.7150563615408072|
+------------------+------------------+------------------+------------------+------------------+

In [5]:
importances = spark.read.json('s3n://amosvoron-sparkify/rfc-downgraded-feature-importances-13h.json')
importances.sort(desc('importance')).show(30, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---------------------+
|feature           |importance           |
+------------------+---------------------+
|NonSongCount      |0.2480559097228076   |
|RollAdvertCount   |0.23603724564133685  |
|RollAdvertPerHour |0.10103617157959105  |
|LogCount          |0.08940420924033478  |
|ThumbsDownCount   |0.0871551632527307   |
|LogoutCount       |0.06786248410464657  |
|TotalSongLength   |0.057630011549019394 |
|SongCount         |0.04673550839547861  |
|UniqueSongCount   |0.030917516916079465 |
|HomePerHour       |0.014774925112209649 |
|AddFriendCount    |0.010120154113186642 |
|AddtoPlaylistCount|0.005767988692350054 |
|SettingsCount     |0.0045027116802285865|
+------------------+---------------------+

In [4]:
importances = spark.read.json('s3n://amosvoron-sparkify/rfc-downgraded-feature-importances-9h.json')
importances.sort(desc('importance')).show(30, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+--------------------+
|feature          |importance          |
+-----------------+--------------------+
|RollAdvertPerHour|0.18078236798390873 |
|NonSongCount     |0.17583812038430244 |
|LogoutCount      |0.1714500683423289  |
|LogCount         |0.1692270590202101  |
|RollAdvertCount  |0.1652819649397765  |
|ThumbsDownCount  |0.0809985802674333  |
|SongCount        |0.03664032098864393 |
|TotalSongLength  |0.014648507389087956|
|AvgSessionGap    |0.005133010684308156|
+-----------------+--------------------+

In [5]:
metrics = spark.read.json('s3n://amosvoron-sparkify/rfc-downgraded-metrics-9h.json')
metrics.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+------------------+------------------+------------------+
|                f1|          accuracy| weightedPrecision|    weightedRecall|               AUC|
+------------------+------------------+------------------+------------------+------------------+
|0.8084889296285297|0.8236351381711975|0.8094240821667913|0.8236351381711975|0.6849522237670034|
+------------------+------------------+------------------+------------------+------------------+

In [1]:
metrics = spark.read.json('s3n://amosvoron-sparkify/rfc-downgraded-metrics-11h.json')
metrics.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
26,application_1597998755054_0049,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------------------+------------------+------------------+------------------+
|               f1|          accuracy| weightedPrecision|    weightedRecall|               AUC|
+-----------------+------------------+------------------+------------------+------------------+
|0.824815310854039|0.8339698944057515|0.8233021851946406|0.8339698944057515|0.7187978616553737|
+-----------------+------------------+------------------+------------------+------------------+

In [2]:
metrics = spark.read.json('s3n://amosvoron-sparkify/rfc-downgraded-metrics-22-gbt.json')
metrics.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+-----------------+------------------+------------------+
|                f1|          accuracy|weightedPrecision|    weightedRecall|               AUC|
+------------------+------------------+-----------------+------------------+------------------+
|0.8148460890342518|0.8292518535160638|0.816390590460442|0.8292518535160638|0.6938103089160288|
+------------------+------------------+-----------------+------------------+------------------+

In [5]:
importances = spark.read.json('s3n://amosvoron-sparkify/rfc-downgraded-feature-importances-22-gbt.json')
importances.sort(desc('importance')).show(30, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+--------------------+
|feature               |importance          |
+----------------------+--------------------+
|NonSongCount          |0.18225428051214868 |
|RollAdvertCount       |0.16125179100117887 |
|ThumbsDownCount       |0.11385443838163774 |
|RollAdvertPerHour     |0.10344833716370305 |
|HomePerHour           |0.07628027064714468 |
|LogoutPerHour         |0.05444018338962429 |
|ThumbsUpCount         |0.03205718017791122 |
|SettingsPerSessionHour|0.030269793748850922|
|AddFriendCount        |0.02947649723433915 |
|LogoutCount           |0.026412296357132003|
|SaveSettingsPerHour   |0.023826479771596444|
|LogCount              |0.022198980709714326|
|ErrorPerHour          |0.020577080551025927|
|AboutPerHour          |0.019885130344729416|
|AddtoPlaylistCount    |0.019408780267448807|
|AboutCount            |0.017095098611739825|
|SettingsCount         |0.01568735139304455 |
|ErrorPerSessionHour   |0.0148550761173494  |
|TotalSongLength       |0.01104548

In [1]:
metrics = spark.read.json('s3n://amosvoron-sparkify/rfc-downgraded-metrics-22h-gbt.json')
metrics.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
28,application_1597998755054_0056,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+------------------+------------------+------------------+
|                f1|          accuracy| weightedPrecision|    weightedRecall|               AUC|
+------------------+------------------+------------------+------------------+------------------+
|0.8187400219006676|0.8281285104470906|0.8167142782328922|0.8281285104470905|0.7108387931871132|
+------------------+------------------+------------------+------------------+------------------+

In [4]:
importances = spark.read.json('s3n://amosvoron-sparkify/rfc-downgraded-feature-importances-22-gbt.json')
importances.sort(desc('importance')).show(30, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+--------------------+
|feature               |importance          |
+----------------------+--------------------+
|NonSongCount          |0.18225428051214868 |
|RollAdvertCount       |0.16125179100117887 |
|ThumbsDownCount       |0.11385443838163774 |
|RollAdvertPerHour     |0.10344833716370305 |
|HomePerHour           |0.07628027064714468 |
|LogoutPerHour         |0.05444018338962429 |
|ThumbsUpCount         |0.03205718017791122 |
|SettingsPerSessionHour|0.030269793748850922|
|AddFriendCount        |0.02947649723433915 |
|LogoutCount           |0.026412296357132003|
|SaveSettingsPerHour   |0.023826479771596444|
|LogCount              |0.022198980709714326|
|ErrorPerHour          |0.020577080551025927|
|AboutPerHour          |0.019885130344729416|
|AddtoPlaylistCount    |0.019408780267448807|
|AboutCount            |0.017095098611739825|
|SettingsCount         |0.01568735139304455 |
|ErrorPerSessionHour   |0.0148550761173494  |
|TotalSongLength       |0.01104548

In [1]:
metrics = spark.read.json('s3n://amosvoron-sparkify/rfc-downgraded-metrics-12h-gbt.json')
metrics.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
29,application_1597998755054_0058,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+------------------+------------------+------------------+
|                f1|          accuracy| weightedPrecision|    weightedRecall|               AUC|
+------------------+------------------+------------------+------------------+------------------+
|0.8216342673718903|0.8290271849022691|0.8190860749874386|0.8290271849022692|0.7201242773554035|
+------------------+------------------+------------------+------------------+------------------+

In [2]:
importances = spark.read.json('s3n://amosvoron-sparkify/rfc-downgraded-feature-importances-22-gbt.json')
importances.sort(desc('importance')).show(30, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'desc' is not defined
Traceback (most recent call last):
NameError: name 'desc' is not defined

